In [19]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import datetime
import numpy as np
import pandas as pd


class Calendar(object):

    WEEKDAY_MAP = {
        1: 'Monday',
        2: 'Tuesday',
        3: 'Wednesday',
        4: 'Thursday',
        5: 'Friday',
        6: 'Saturday',
        7: 'Sunday',
    }

    CALENDAR_KEYS = [
        'date',
        'year',
        'month',
        'day',
        'weekday',
        'weekday_name',
        'weeknumber',
        'is_weekend',
        'is_business_day',
        'is_holiday',
        'holiday_name',
    ]

    def __init__(self, start_date_string, end_date_string,
        include_holidays=False, holidays={}, format='%Y-%m-%d'):
        """
        Initialize with a start and end date string with a default format
        of %Y-%m-%d (YYYY-MM-DD). This can be changed with the
        format argument.

        include_holidays is a boolean if you want to include holidays in the calendar.
        """
        self.start_date = datetime.datetime.strptime(start_date_string, format)
        self.end_date = datetime.datetime.strptime(end_date_string, format)
        self.num_days = (self.end_date - self.start_date).days
        self.include_holidays = include_holidays
        self.CALENDAR_DATA = self._gen_calendar_data_template()
        self.holidays = holidays

    def _gen_calendar_data_template(self):
        """
        Generates a calendar data template with empty lists assigned to each
        key in `self.CALENDAR_KEYS`
        """
        return { k:[] for k in self.CALENDAR_KEYS }

    def _gen_dates(self):
        for day_num in range(self.num_days):
            yield self.start_date + datetime.timedelta(days=day_num)

    def _is_weekend(self, weekday_index):
        """
        If it's a weekend, return True.
        """
        return True if weekday_index in (6, 7) else False

    def _is_holiday(self, datetime_object):
        """
        takes a datetime date object and returns True if it is a public
        holiday on the date.
        """
        date_key = datetime.datetime.strftime(datetime_object, '%Y-%m-%d')
        holiday = self.holidays.get(date_key)
        if holiday is None:
            #return (False, np.nan)
            return (False, "")
        return (True, holiday)

    def _is_business_day(self, weekday_index, datetime_object):
        """
        Returns True for M-F providing they're not holidays.
        """
        if self.include_holidays is True:
            return not self._is_weekend(weekday_index) and not self._is_holiday(datetime_object)[0]
        return not self._is_weekend(weekday_index)

    def generate(self, dest=None, sep=',', reset=False):
        """
        Generates the calendar. If reset is True it will reset the cal.

        If destination is provided it will save the calendar. You can use
        the sep argument to define the delimiter.
        """
        if reset:
            self.CALENDAR_DATA = self._gen_calendar_data_template()

        for date in self._gen_dates():
            year, weeknumber, weekday = date.isocalendar()
            self.CALENDAR_DATA['date'].append('%s' % date.date())
            self.CALENDAR_DATA['year'].append(year)
            self.CALENDAR_DATA['month'].append(date.month)
            self.CALENDAR_DATA['day'].append(date.day)
            self.CALENDAR_DATA['weekday'].append(weekday)
            self.CALENDAR_DATA['weekday_name'].append(self.WEEKDAY_MAP[weekday])
            self.CALENDAR_DATA['weeknumber'].append(weeknumber)
            if self._is_weekend(weekday):
                weekend_status = "1"
            else:
                weekend_status = "0"
            self.CALENDAR_DATA['is_weekend'].append(weekend_status)

            # Handle holidays
            if self.include_holidays:
                is_holiday, holiday_name = self._is_holiday(date)
                #after true/false replace to 1/0
                if is_holiday:
                    is_holiday = "1"
                else:
                    is_holiday = "0"
            else:
                is_holiday = np.nan
                holiday_name = np.nan
                
            
            if self._is_business_day(weekday, date):
                business_status = "1"
            else:
                business_status = "0"
                
            self.CALENDAR_DATA['is_holiday'].append(is_holiday)
            self.CALENDAR_DATA['holiday_name'].append(holiday_name)
            self.CALENDAR_DATA['is_business_day'].append(business_status)

        calendar = pd.DataFrame(self.CALENDAR_DATA)[self.CALENDAR_KEYS]

        if dest is None:
            return calendar

        return calendar.to_csv(dest, sep=sep, index=False)

In [20]:
holiday = {'2017-10-28': ' Cumhuriyet Bayramı',
 '2017-10-29': ' Cumhuriyet Bayramı',
 '2018-10-28': ' Cumhuriyet Bayramı',
 '2018-10-29': ' Cumhuriyet Bayramı',
 '2019-10-28': ' Cumhuriyet Bayramı',
 '2019-10-29': ' Cumhuriyet Bayramı',
 '2020-10-28': ' Cumhuriyet Bayramı',
 '2020-10-29': ' Cumhuriyet Bayramı',
 '2021-10-28': ' Cumhuriyet Bayramı',
 '2021-10-29': ' Cumhuriyet Bayramı',
 '2022-10-28': ' Cumhuriyet Bayramı',
 '2022-10-29': ' Cumhuriyet Bayramı',
 '2017-07-15': ' Demokrasi ve Milli Birlik Günü',
 '2018-07-15': ' Demokrasi ve Milli Birlik Günü',
 '2019-07-15': ' Demokrasi ve Milli Birlik Günü',
 '2020-07-15': ' Demokrasi ve Milli Birlik Günü',
 '2021-07-15': ' Demokrasi ve Milli Birlik Günü',
 '2022-07-15': ' Demokrasi ve Milli Birlik Günü',
 '2017-08-31': ' Kurban Bayramı Arifesi',
 '2017-09-01': ' Kurban Bayramı',
 '2017-09-02': ' Kurban Bayramı',
 '2017-09-03': ' Kurban Bayramı',
 '2017-09-04': ' Kurban Bayramı',
 '2018-08-20': ' Kurban Bayramı Arifesi',
 '2018-08-21': ' Kurban Bayramı',
 '2018-08-22': ' Kurban Bayramı',
 '2018-08-23': ' Kurban Bayramı',
 '2018-08-24': ' Kurban Bayramı',
 '2019-08-10': ' Kurban Bayramı Arifesi',
 '2019-08-11': ' Kurban Bayramı',
 '2019-08-12': ' Kurban Bayramı',
 '2019-08-13': ' Kurban Bayramı',
 '2019-08-14': ' Kurban Bayramı',
 '2020-07-30': ' Kurban Bayramı Arifesi',
 '2020-07-31': ' Kurban Bayramı',
 '2020-08-01': ' Kurban Bayramı',
 '2020-08-02': ' Kurban Bayramı',
 '2020-08-03': ' Kurban Bayramı',
 '2021-07-19': ' Kurban Bayramı Arifesi',
 '2021-07-20': ' Kurban Bayramı',
 '2021-07-21': ' Kurban Bayramı',
 '2021-07-22': ' Kurban Bayramı',
 '2021-07-23': ' Kurban Bayramı',
 '2022-07-08': ' Kurban Bayramı Arifesi',
 '2022-07-09': ' Kurban Bayramı',
 '2022-07-10': ' Kurban Bayramı',
 '2022-07-11': ' Kurban Bayramı',
 '2022-07-12': ' Kurban Bayramı',
 '2017-08-30': ' Zafer Bayramı',
 '2018-08-30': ' Zafer Bayramı',
 '2019-08-30': ' Zafer Bayramı',
 '2020-08-30': ' Zafer Bayramı',
 '2021-08-30': ' Zafer Bayramı',
 '2022-08-30': ' Zafer Bayramı',
 '2017-05-19': " Atatürk'ü Anma Gençlik ve Spor Bayramı",
 '2018-05-19': " Atatürk'ü Anma Gençlik ve Spor Bayramı",
 '2019-05-19': " Atatürk'ü Anma Gençlik ve Spor Bayramı",
 '2020-05-19': " Atatürk'ü Anma Gençlik ve Spor Bayramı",
 '2021-05-19': " Atatürk'ü Anma Gençlik ve Spor Bayramı",
 '2022-05-19': " Atatürk'ü Anma Gençlik ve Spor Bayramı",
 '2017-06-24': ' Ramazan Bayramı Arifesi',
 '2017-06-25': ' Ramazan Bayramı',
 '2017-06-26': ' Ramazan Bayramı',
 '2017-06-27': ' Ramazan Bayramı',
 '2018-06-14': ' Ramazan Bayramı Arifesi',
 '2018-06-15': ' Ramazan Bayramı',
 '2018-06-16': ' Ramazan Bayramı',
 '2018-06-17': ' Ramazan Bayramı',
 '2019-06-04': ' Ramazan Bayramı Arifesi',
 '2019-06-05': ' Ramazan Bayramı',
 '2019-06-06': ' Ramazan Bayramı',
 '2019-06-07': ' Ramazan Bayramı',
 '2020-05-23': ' Ramazan Bayramı Arifesi',
 '2020-05-24': ' Ramazan Bayramı',
 '2020-05-25': ' Ramazan Bayramı',
 '2020-05-26': ' Ramazan Bayramı',
 '2021-05-12': ' Ramazan Bayramı  Arifesi',
 '2021-05-13': ' Ramazan Bayramı',
 '2021-05-14': ' Ramazan Bayramı',
 '2021-05-15': ' Ramazan Bayramı',
 '2022-05-01': ' Emek ve Dayanışma Günü',
 '2022-05-02': ' Ramazan Bayramı',
 '2022-05-03': ' Ramazan Bayramı',
 '2022-05-04': ' Ramazan Bayramı',
 '2017-05-01': ' Emek ve Dayanışma Günü',
 '2018-05-01': ' Emek ve Dayanışma Günü',
 '2019-05-01': ' Emek ve Dayanışma Günü',
 '2020-05-01': ' Emek ve Dayanışma Günü',
 '2021-05-01': ' Emek ve Dayanışma Günü',
 '2017-04-23': ' Ulusal Egemenlik ve Çocuk Bayramı',
 '2018-04-23': ' Ulusal Egemenlik ve Çocuk Bayramı',
 '2019-04-23': ' Ulusal Egemenlik ve Çocuk Bayramı',
 '2020-04-23': ' Ulusal Egemenlik ve Çocuk Bayramı',
 '2021-04-23': ' Ulusal Egemenlik ve Çocuk Bayramı',
 '2022-04-23': ' Ulusal Egemenlik ve Çocuk Bayramı',
 '2017-01-01': ' Yılbaşı',
 '2018-01-01': ' Yılbaşı',
 '2019-01-01': ' Yılbaşı',
 '2020-01-01': ' Yılbaşı',
 '2021-01-01': ' Yılbaşı',
 '2022-01-01': ' Yılbaşı'}

In [21]:
start_date = '2017-01-01'
end_date = '2022-01-01'

calendar = Calendar(start_date, end_date, include_holidays=True, holidays=holiday)

data_frame = calendar.generate()
data_frame.head()

,date,year,month,day,weekday,weekday_name,weeknumber,is_weekend,is_business_day,is_holiday,holiday_name
0,2017-01-01,2016,1,1,7,Sunday,52,1,0,1,Yılbaşı
1,2017-01-02,2017,1,2,1,Monday,1,0,1,0,
2,2017-01-03,2017,1,3,2,Tuesday,1,0,1,0,
3,2017-01-04,2017,1,4,3,Wednesday,1,0,1,0,
4,2017-01-05,2017,1,5,4,Thursday,1,0,1,0,


In [22]:
data_frame.to_csv('0-1.csv', encoding='utf-8')